In [ ]:
!pip install diffusers transformers accelerate safetensors gradio rembg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()



In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# Load the model (v1.5 example)
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,   # faster inference on GPU
    use_safetensors=True
)

# Move pipeline to GPU
pipe = pipe.to("gpu")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

safety_checker/model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import os
from PIL import Image

def generate_ad(prompt, out_dir="/content/drive/MyDrive/ad_outputs/", n=1, steps=20):
    os.makedirs(out_dir, exist_ok=True)
    paths = []
    for i in range(n):
        out = pipe(prompt, num_inference_steps=steps, guidance_scale=7.5)
        img = out.images[0]
        path = os.path.join(out_dir, f"ad_{i}.png")
        img.save(path)
        paths.append(path)
    return paths


In [ ]:
prompt = "Modern banner ad for a smartwatch sale, clean layout, white background, product focus"
files = generate_ad(prompt, n=3, steps=25)
files


In [ ]:
!pip install gradio

In [ ]:

import gradio as gr

def gradio_generate(prompt):
    files = generate_ad(prompt, n=1)
    return files[0]

gr.Interface(fn=gradio_generate, inputs="text", outputs="image").launch()
